In [1]:
import transformers
import torch
from gpt2_modified import *
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer, TrainingArguments, GPT2Config, Trainer
model_path = '/mntcephfs/data/ruoyusun/common_dirs/gpt2-small'
tokenizer = AutoTokenizer.from_pretrained(model_path)

/mntcephfs/lab_data/hanyizhou/anaconda/pe/lib/python3.11/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/mntcephfs/lab_data/hanyizhou/anaconda/pe/lib/python3.11/site-packages/torch/cuda/__init__.py:749: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /opt/conda/conda-bld/pytorch_1716905969073/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count


In [2]:
#dataset = load_dataset('./dataset/testset/validationset')['train']
dataset = load_from_disk('/mntcephfs/data/ruoyusun/hanyizhou/data/c4_512')
dataset

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 890793
})

### 制作50个句子, 其中每个句子长度都是用50个短句子拼接起来的.

In [3]:
from tqdm import tqdm
import torch
ds = {}
for length in [512, 1024, 2048, 3072, 4096, 6144, 8192]:
    ds[length] = []
for i in tqdm(range(0,2500,50)): 
    data = sum(dataset[i:50+i]['input_ids'],[])
    for length in [512, 1024, 2048, 3072, 4096, 6144, 8192]:
        input_ids = data[:length]
        ds[length].append(input_ids)


100%|██████████| 50/50 [00:00<00:00, 77.19it/s]


In [17]:
len(ds[8192][10])

8192

ds中的数据结构为:


length [512, 1024, 2048, 3072, 4096, 6144, 8192]


list: sentence[0-49]

### 制作三类模型

In [3]:
#模型config设置
config = GPT2Config()

config.resid_pdrop = 0.0
config.embd_pdrop = 0.0
config.attn_pdrop = 0.0
config.vocab_size = 50258

In [ ]:
model_fire = {}
model_cope = {}
model_mix = {}
#model_change = {}
for length in [512, 1024, 2048, 3072, 4096, 6144, 8192]:
    config.n_positions = length
    fire = GPT2_FIRE.from_pretrained('./output/fire/checkpoint-41250', config = config, ignore_mismatched_sizes=True)
    fire.transformer.wpe = torch.nn.Embedding(length,768)
    fire.transformer.wpe.weight = torch.nn.Parameter(torch.zeros(length,768))
    cope = GPT2_CoPE.from_pretrained('./output/cope/checkpoint-41250', config = config, ignore_mismatched_sizes=True,n_posmax = 384)
    mix = GPT2_FireWithCoPE.from_pretrained('./output/fire_with_cope/checkpoint-41250', config = config, ignore_mismatched_sizes=True)
    cope.transformer.wpe = torch.nn.Embedding(length,768)
    cope.transformer.wpe.weight = torch.nn.Parameter(torch.zeros(length,768))
    mix.transformer.wpe = torch.nn.Embedding(length,768)
    mix.transformer.wpe.weight = torch.nn.Parameter(torch.zeros(length,768))
    change = mix
    keys = fire.transformer._modules.keys()
    for key in keys:
        if key != 'h':
            change.transformer._modules[key] = fire.transformer._modules[key]
        else:
            for i in range(12):
                change.transformer.h[i].attn.c_attn = fire.transformer.h[i].attn.c_attn
                change.transformer.h[i].attn.c_proj = fire.transformer.h[i].attn.c_proj
                
    change.lm_head = fire.lm_head
    model_fire[length] = fire
    model_cope[length] = cope
    model_mix[length] = mix
    model_change[length] = change

In [40]:
model_change[512]

GPT2_FireWithCoPE(
  (transformer): GPT2Model(
    (wte): Embedding(50258, 768)
    (wpe): Embedding(512, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block_FireWithCoPE(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention_FireWithCoPE(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (fire_with_cope): CoPEWithFIRE(
            (input): Linear(in_features=1, out_features=32, bias=True)
            (mlp_list): ModuleList(
              (0-11): 12 x Sequential(
                (0): Linear(in_features=1, out_features=32, bias=True)
                (1): ReLU()
                (2): Linear(in_features=32, out_features=1, bias=True)
              )
            )
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT

### 模型测试

In [ ]:
acc_fire, acc_cope, acc_mix = {}, {}, {}
for length in [512, 1024, 2048, 3072, 4096, 6144, 8192]:
    acc_fire[length] = []
    acc_cope[length] = []
    acc_mix[length] = []
for length in tqdm([512, 1024, 2048, 3072, 4096, 6144, 8192]):
    sequence_list = torch.tensor(ds[length])
    fire = model_fire[length]
    cope = model_cope[length]
    mix = model_mix[length]
    fire_output = fire(input_ids = sequence_list, labels = sequence_list)
    cope_output = cope(input_ids = sequence_list, labels = sequence_list)
    mix_output = mix(input_ids = sequence_list, labels = sequence_list)
    acc_fire[length].append(fire_output.loss)
    acc_cope[length].append(cope_output.loss)
    acc_mix[length].append(mix_output.loss)

In [1]:
import numpy as np
import torch

In [2]:
load_mix = np.load('./output/acc_mix.npy',allow_pickle=True).item()
load_fire= np.load('./output/acc_fire.npy',allow_pickle=True).item()
load_cope= np.load('./output/acc_cope.npy',allow_pickle=True).item()
load_change = np.load('./output/acc_change.npy',allow_pickle=True).item()
for key in load_mix.keys():
    load_mix[key] = torch.tensor(load_mix[key]).mean().numpy()
    load_fire[key] = torch.tensor(load_fire[key]).mean().numpy()
    load_cope[key] = torch.tensor(load_cope[key]).mean().numpy()
    load_change[key] = torch.tensor(load_change[key]).mean().numpy()


In [4]:
load_change, load_mix, load_fire, load_cope

({512: array(11.366022, dtype=float32),
  1024: array(11.8661, dtype=float32),
  2048: array(12.179937, dtype=float32),
  3072: array(12.306825, dtype=float32),
  4096: array(12.414685, dtype=float32),
  6144: array(12.526135, dtype=float32),
  8192: array(12.590228, dtype=float32)},
 {512: array(11.366022, dtype=float32),
  1024: array(11.8661, dtype=float32),
  2048: array(12.179937, dtype=float32),
  3072: array(12.306825, dtype=float32),
  4096: array(12.414685, dtype=float32),
  6144: array(12.526135, dtype=float32),
  8192: array(12.590228, dtype=float32)},
 {512: array(3.4227178, dtype=float32),
  1024: array(3.5610778, dtype=float32),
  2048: array(3.641525, dtype=float32),
  3072: array(3.7055151, dtype=float32),
  4096: array(3.786891, dtype=float32),
  6144: array(3.9346197, dtype=float32),
  8192: array(4.0672827, dtype=float32)},
 {512: array(3.6817214, dtype=float32),
  1024: array(3.8474314, dtype=float32),
  2048: array(4.149472, dtype=float32),
  3072: array(4.5055146,